In [90]:
# Functions to load and parse data

def load_data(filename = "../data/day19test.txt"):
    with open(filename, "r") as f:
        input = f.read()

    parts = input.split("\n\n")
    rules = parts[0]
    msgs = parts[1].split("\n")

    rule_dict = {int(rule.split(":")[0]):rule.split(":")[1][1:] for rule in rules.split("\n")}
    rule_dict_parsed = parse_rule_dict(rule_dict)
    return rule_dict_parsed, msgs


def parse_rule_dict(rule_dict: dict):
    rule_dict_parsed = {}
    for n, rule_to_parse in rule_dict.items():
        if "|" in rule_to_parse:
            rule_type = "two_sequences"
            parts = rule_to_parse.split("|")
            value = [[int(value) for value in part.strip().split(" ")] for part in parts]

        elif "a" in rule_to_parse or "b" in rule_to_parse:
            rule_type = "letter"
            value = rule_to_parse.strip('"')

        else:
            rule_type = "sequence"
            value = [int(value) for value in rule_to_parse.strip().split(" ")]

        rule_dict_parsed[n] = (rule_type, value)

    return rule_dict_parsed


In [ ]:
# Puzzle 1 with objects

# Not trees per se, but object-oriented, to easily copy objects.

# Parsing function:
# if a/b: store value, drop doing, move one of todo to doing, stop if todo is empty
# if 1 2 3: put first into doing, rest into todo. 
# if 1 2 | 2 1: copy object, put both firsts into doing, rest into todo. call parsing functions of both

class RuleSet:
    """
    Main class containing all of the rules and their results.
    """
    def __init__(self, rule_dict):
        self.rule_dict = rule_dict
        self.rules = []

        self.parse()

    def parse(self):
        """Create initial rule, also refer back to RuleSet to be able to add new rules dynamically."""
        initial_rule = Rule(self.rule_dict, ruleset=self)
        self.rules.append(initial_rule)

    def get_results(self):
        """Return list of results."""
        return [rule.result for rule in self.rules]

class Rule():
    """
    Class to recursively parse the ruleset, creates new instance if a split is encountered.
    """
    def __init__(self, rule_dict, todo=[], result="", ruleset=None, start_n=0):
        self.rule_dict = rule_dict
        self.todo = todo
        self.result = result
        self.ruleset = ruleset

        self.parse(start_n)

    def parse(self, n=0):
        rule_to_parse = self.rule_dict[n]
        rule_type = rule_to_parse[0]
        rule_value = rule_to_parse[1]

        if rule_type == "two_sequences":
            queue = rule_value[1]
            todo = queue + self.todo
            doing = todo.pop(0)
            rule_new = Rule(self.rule_dict, todo, self.result, self.ruleset, doing)
            self.ruleset.rules.append(rule_new)

            self.parse_sequence(rule_value[0])
            
        if rule_type == "sequence":
            self.parse_sequence(rule_value)

        if rule_type == "letter":
            if rule_value == "a":
                self.result += "a"
            if rule_value == "b":
                self.result += "b"
            if self.todo:
                self.parse(self.todo.pop(0))
        return

    def parse_sequence(self, queue: list):
        """Parse 1 2 3"""
        self.todo = queue + self.todo
        doing = self.todo.pop(0)
        self.parse(doing)

def solve_puzzle(filename="../data/day19.txt"):
    rule_dict_parsed, msgs = load_data(filename)
    rules = RuleSet(rule_dict_parsed)
    allowed_messages = rules.get_results()

    n = 0
    for msg in msgs:
        if msg in allowed_messages:
            n += 1
    print(str(n) + " out of " + str(len(msgs)) + " are allowed")
    return n

solve_puzzle("../data/day19.txt")

In [91]:
# Puzzle 1 with regex
import re
from pprint import pprint


def parse_rule(rule_dict_parsed, n=0):
    rule_to_parse = rule_dict_parsed[n]
    rule_type = rule_to_parse[0]
    rule_value = rule_to_parse[1]

    if rule_type == "two_sequences":
        result_list_0 = [parse_rule(rule_dict_parsed, value) for value in rule_value[0]]
        result_list_1 = [parse_rule(rule_dict_parsed, value) for value in rule_value[1]]
        result = "(" + "".join(result_list_0) + "|" + "".join(result_list_1) + ")"
    if rule_type == "sequence":
        result_list = [parse_rule(rule_dict_parsed, value) for value in rule_value]
        result = "".join(result_list)
    if rule_type == "letter":
        result = rule_value
    return result


def solve_with_regex(rule_dict_parsed, msgs, parser):
    regex_result_str = parser(rule_dict_parsed)
    regex_str = regex.compile("^" + regex_result_str + "$")

    #print(regex_result_str)

    n = 0
    for msg in msgs:
        if regex_str.match(msg):
            n += 1
    print(str(n) + " out of " + str(len(msgs)) + " messages are allowed")
    return n


rule_dict_parsed, msgs = load_data("../data/day19.txt")
n = solve_with_regex(rule_dict_parsed, msgs, parse_rule)

124 out of 368 messages are allowed


In [92]:
# Puzzle 2 with regex

# Parts of it can be repeated
# 8: 42 | 42 8 > first part repeated n times
# 11: 42 31 | 42 11 31 > nested parts

# 42 31
# 42 42 31 31
# 42 42 42 31 31 31
# 42 42 42 42 31 31 31 31
# ...
# so only an equal number!
# ab
# aabb
# aaabbb
# https://stackoverflow.com/questions/17053438/use-regex-to-match-axbx-where-x-is-the-number-of-times-a-b-appear
# Works in regex101 but needs numbered reference to group which gets difficult (maybe named reference?)
# ^(a(?-1)?b)$ matches ab, aabb, aaabbb in regex101 but does not work in Python re or regex.
# Use named groups finally, and Python regex for that.

# New parser function
def parse_rule_2(rule_dict_parsed, n=0):
    rule_to_parse = rule_dict_parsed[n]
    rule_type = rule_to_parse[0]
    rule_value = rule_to_parse[1]

    if n == 8:
        result_42 = parse_rule_2(rule_dict_parsed, 42)
        result = "(" + result_42 + ")+"
    elif n == 11:
        result_42 = parse_rule_2(rule_dict_parsed, 42)
        result_31 = parse_rule_2(rule_dict_parsed, 31)
        result = "(?P<group>" + result_42 + "(?P>group)?" + result_31 + ")"
    else:
        if rule_type == "two_sequences":
            result_list_0 = [parse_rule_2(rule_dict_parsed, value) for value in rule_value[0]]
            result_list_1 = [parse_rule_2(rule_dict_parsed, value) for value in rule_value[1]]
            result = "(" + "".join(result_list_0) + "|" + "".join(result_list_1) + ")"
        if rule_type == "sequence":
            result_list = [parse_rule_2(rule_dict_parsed, value) for value in rule_value]
            result = "".join(result_list)
        if rule_type == "letter":
            result = rule_value
    return result


# Load rule dictionary
rule_dict_parsed, msgs = load_data("../data/day19.txt")

# Unchanged example
n = solve_with_regex(rule_dict_parsed, msgs, parse_rule)
print(n)

# Changed example
n = solve_with_regex(rule_dict_parsed, msgs, parse_rule_2)
print(n)


124 out of 368 messages are allowed
124
228 out of 368 messages are allowed
228


In [94]:
import regex

# Works
regex_str = regex.compile("^(a(?1)?b)$") 
print(regex_str.match("aabb"))

# Does not work
regex_str = regex.compile("^(x)(a(?1)?b)$") 
print(regex_str.match("xaabb"))

# Does not work
#regex_str = regex.compile("^(a(?-1)?b)$") 
#regex_str.match("aabb")

# Works
regex_str = regex.compile("^(?P<bla>a(?P>bla)?b)$") 
print(regex_str.match("aaabbb"))

# Works
regex_str = regex.compile("^((x)(?P<group>a(?P>group)?b))(y)$") 
print(regex_str.match("xaaabbby"))


<regex.Match object; span=(0, 4), match='aabb'>
None
<regex.Match object; span=(0, 6), match='aaabbb'>
<regex.Match object; span=(0, 8), match='xaaabbby'>
